<a href="https://colab.research.google.com/github/Blue22796/Auto-searcher/blob/main/mount_onedrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Tutorial adapted from https://www.youtube.com/watch?v=U6YPgARhRzA

#@markdown The first step is to download a copy of rclone from https://downloads.rclone.org **in your system**, unzip it and execute in a terminal `rclone authorize "onedrive"`, log into your account and copy the access token. Then run the cell to install `rclone` in Colab.

!wget https://downloads.rclone.org/v1.56.0/rclone-v1.56.0-linux-amd64.deb
!apt install ./rclone-v1.56.0-linux-amd64.deb

In [ ]:
#@markdown Run this cell, and in rclone config, select `n` for new remote, name it `onedrive` and check the number in the list corresponding to onedrive (`26` in the current version). Then press enter for `client_id` and for `client_secret`, then `n` to avoid avanced config and `n` for auto config.
#@markdown Then paste the access token code generated in your machine, and select number 1 for "Onedrive personal" or "Onedrive business"
#@markdown Then, in "found drives" check that the configuration is ok, depending on the one you want to mount. Then select `y` twice and `q` to quit the configuration.
!rclone config

In [ ]:
#@markdown Then, you can mount the drive in `\content\gdrive` to use it seamlessly in the ZeroCostDL4Mic notebooks
import os
import sys
folder_name = "gdrive" #@param {type:"string"}
folder=str('/content/'+folder_name)
print("Mounted in",folder)
os.mkdir(folder)
os.system("nohup rclone --vfs-cache-mode writes mount onedrive: %s &" % folder)

In [2]:
!python -m pip install --upgrade pip setuptools wheel

!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-libtorrent is already the newest version (2.0.5-5).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [7]:
%%capture
!python -m pip install --upgrade pip setuptools wheel

!python -m pip install lbry-libtorrent
!apt install python3-libtorrent
import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

In [5]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [8]:
params = {"save_path": "/content/sample_data/Torrent"}
#params = {"save_path" : "/content/drive/MyDrive/Torrent"}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )


Enter Magnet Link Or Type Exit: magnet:?xt=urn:btih:8F7B8073991969F9220A6624530FC13C619EF17D&dn=Yakuza+3+Remastered+%28MULTi4%29+%5BFitGirl+Repack%5D&tr=udp%3A%2F%2F46.148.18.250%3A2710&tr=udp%3A%2F%2Fopentor.org%3A2710&tr=udp%3A%2F%2Ftracker.coppersurfer.tk%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=http%3A%2F%2Ftracker.dler.org%3A6969%2Fannounce&tr=udp%3A%2F%2Ftracker.leechers-paradise.org%3A6969%2Fannounce&tr=udp%3A%2F%2F9.rarbg.me%3A2730%2Fannounce&tr=udp%3A%2F%2F9.rarbg.to%3A2770%2Fannounce&tr=udp%3A%2F%2Ftracker.pirateparty.gr%3A6969%2Fannounce&tr=http%3A%2F%2Fretracker.local%2Fannounce&tr=http%3A%2F%2Fretracker.ip.ncnet.ru%2Fannounce&tr=udp%3A%2F%2Fexodus.desync.com%3A6969%2Fannounce&tr=udp%3A%2F%2Fipv4.tracker.harry.lu%3A80%2Fannounce&tr=udp%3A%2F%2Fopen.stealth.si%3A80%2Fannounce&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=http%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannou

In [9]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)


FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

Yakuza 3 Remastered [FitGirl Repack] complete
Star Ocean - The Divine Force [FitGirl Repack] complete


In [ ]:
def split_binary_file(input_file, chunk_size= 3 * 1024 * 1024 * 1024):  # 10 GB
    with open(input_file, "rb") as f:
        part = 1
        while chunk := f.read(chunk_size):  # Read chunk of at most 10 GB
            output_file = f"{input_file}.part{part}"
            with open(output_file, "wb") as out:
                out.write(chunk)
            print(f"Created {output_file}")
            part += 1


In [ ]:
# Example usage
split_binary_file("/content/sample_data/Torrent/Monster Hunter World - Iceborne [FitGirl Repack]/fg-01.bin")


Created /content/sample_data/Torrent/Monster Hunter World - Iceborne [FitGirl Repack]/fg-01.bin.part1
Created /content/sample_data/Torrent/Monster Hunter World - Iceborne [FitGirl Repack]/fg-01.bin.part2
Created /content/sample_data/Torrent/Monster Hunter World - Iceborne [FitGirl Repack]/fg-01.bin.part3
Created /content/sample_data/Torrent/Monster Hunter World - Iceborne [FitGirl Repack]/fg-01.bin.part4
Created /content/sample_data/Torrent/Monster Hunter World - Iceborne [FitGirl Repack]/fg-01.bin.part5
Created /content/sample_data/Torrent/Monster Hunter World - Iceborne [FitGirl Repack]/fg-01.bin.part6
Created /content/sample_data/Torrent/Monster Hunter World - Iceborne [FitGirl Repack]/fg-01.bin.part7
Created /content/sample_data/Torrent/Monster Hunter World - Iceborne [FitGirl Repack]/fg-01.bin.part8
Created /content/sample_data/Torrent/Monster Hunter World - Iceborne [FitGirl Repack]/fg-01.bin.part9
Created /content/sample_data/Torrent/Monster Hunter World - Iceborne [FitGirl Repa

In [10]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.oauth2 import service_account

def upload_to_drive(file_path, folder_id=None):
    """Uploads a file to Google Drive."""
    SCOPES = ['https://www.googleapis.com/auth/drive.file']
    SERVICE_ACCOUNT_FILE = '/content/sample_data/Torrent/glossy-flow-451423-k9-56b7c048a172.json'  # Update with your service account JSON file

    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('drive', 'v3', credentials=creds)

    file_metadata = {'name': file_path.split('/')[-1]}
    if folder_id:
        file_metadata['parents'] = [folder_id]

    media = MediaFileUpload(file_path, resumable=True)
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    print(f'File uploaded successfully, File ID: {file.get("id")}')
    return file.get("id")


In [11]:
folders = ['1nB16QQXIhPrCPDuzhVbwTy3fcyMbkT0U', '1EUYpoJti2QmJ0Yd78KnpyLSqK-Izlmp3', '1uu0nED0FmTSDI1WQw4Ypzq0T2ZjXs0xn', '1xBc8aVloW5mVIT2hs-IrVXRYpfxPbMBr', '1sP5h6JmEUat1JxQ9Pkf0tbnLeXcBuaJB']
for i in range(1):
  fil = '/content/sample_data/Torrent/Yakuza 3 Remastered [FitGirl Repack]/fg-01.bin'
  upload_to_drive(fil, folders[0])
  print(fil[-6:])

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/Torrent/glossy-flow-451423-k9-56b7c048a172.json'

In [14]:
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials

# Replace with your token or OAuth credentials
SCOPES = ['https://www.googleapis.com/auth/drive.file']
SERVICE_ACCOUNT_FILE = '/content/sample_data/Torrent/glossy-flow-451423-k9-56b7c048a172.json'  # Update with your service account JSON file
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

drive_service = build("drive", "v3", credentials=creds)

def list_files():
    results = drive_service.files().list(fields="files(id, name)").execute()
    files = results.get("files", [])

    if not files:
        print("No files found.")
    else:
        for file in files:
            print(f"Found file: {file['name']} (ID: {file['id']})")

list_files()

Found file: fg-optional-japanese.bin (ID: 1RO9o2NbE7JB1kjUOX2adnTPKxf3u4AvZ)
Found file: fg-optional-french.bin (ID: 1UCVNGwjqx0ISEldrvv_eS7HL4V1qq3Ia)
Found file: fg-optional-credits.bin (ID: 1DaLK38JyiIMbTw3DSeimKgzgZ5ix41b9)
Found file: fg-optional-bonus-content.bin (ID: 1qtFRV4eG-3TRA3aZht4rnb3aqaMnSOB4)
Found file: ymnfg-03.bin (ID: 162hZxCA32f81oxFbog_mM8xo6ulLiSSM)
Found file: ymnfg-02.bin (ID: 1vqySxCQTCJbBFg2CDJ7ZAosICd2ERkR8)
Found file: ymnfg-01.bin (ID: 1jnGwI4qtc179KND4hUl3A_FvIEpWVzag)
Found file: yfg-03.bin (ID: 19ao6Rggy_WtgN9FnlWwVTa-uiytKexJ9)
Found file: fg-04.bin (ID: 1nr1qnUVg3eern0YOpbSMMWAozowPPoIc)
Found file: fg-02.bin (ID: 1w24Sjs5aAEJ46_aLq--QjtwVLww3V2nP)
Found file: fg-01.bin (ID: 1RcfhIcDz2dtgocnk_XAmNrjsDMC0r5sj)
Found file: fg-03.bin (ID: 17FVqG6tlq6iFBtlroHlnI1vviSIONGPc)


In [15]:
def delete_all_files():
    results = drive_service.files().list(fields="files(id, name)").execute()
    files = results.get("files", [])

    if not files:
        print("No files to delete.")
        return

    for file in files:
        try:
            drive_service.files().delete(fileId=file["id"]).execute()
            print(f"Deleted: {file['name']} (ID: {file['id']})")
        except Exception as e:
            print(f"Failed to delete {file['name']}: {e}")

delete_all_files()

Deleted: fg-optional-japanese.bin (ID: 1RO9o2NbE7JB1kjUOX2adnTPKxf3u4AvZ)
Deleted: fg-optional-french.bin (ID: 1UCVNGwjqx0ISEldrvv_eS7HL4V1qq3Ia)
Deleted: fg-optional-credits.bin (ID: 1DaLK38JyiIMbTw3DSeimKgzgZ5ix41b9)
Deleted: fg-optional-bonus-content.bin (ID: 1qtFRV4eG-3TRA3aZht4rnb3aqaMnSOB4)
Deleted: ymnfg-03.bin (ID: 162hZxCA32f81oxFbog_mM8xo6ulLiSSM)
Deleted: ymnfg-02.bin (ID: 1vqySxCQTCJbBFg2CDJ7ZAosICd2ERkR8)
Deleted: ymnfg-01.bin (ID: 1jnGwI4qtc179KND4hUl3A_FvIEpWVzag)
Deleted: yfg-03.bin (ID: 19ao6Rggy_WtgN9FnlWwVTa-uiytKexJ9)
Deleted: fg-04.bin (ID: 1nr1qnUVg3eern0YOpbSMMWAozowPPoIc)
Deleted: fg-02.bin (ID: 1w24Sjs5aAEJ46_aLq--QjtwVLww3V2nP)
Deleted: fg-01.bin (ID: 1RcfhIcDz2dtgocnk_XAmNrjsDMC0r5sj)
Deleted: fg-03.bin (ID: 17FVqG6tlq6iFBtlroHlnI1vviSIONGPc)
